# PageRank (Random Walk with Restart)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# networkx python에서 graph를 다루는 library, graph mining에 집중

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import os

# edge는 page간의 hyperlink
# page가 graph

## Data import

In [ ]:
basicpath = '/content/drive/MyDrive/Datascience_colab/14주차/'

file = 'facebook.edges'
g = nx.read_edgelist(os.path.join(basicpath, file), create_using=nx.DiGraph)

In [ ]:
g.nodes

In [ ]:
g.edges

In [ ]:
g.size()

## Data 확인

In [ ]:
plt.figure(figsize=[20, 15])
np.random.seed(5)
nx.draw_networkx(g, with_labels=True, node_size=1000, font_size=10, font_color='white', arrowsize=10)
plt.show()

# 어떤 page가 가장 중요하냐?
# 래리 페이지는 random web surfer들이 가장 많이 방문한 페이지가 가장 중요한 페이지다
# 그 페이지에 가장 많은 정보가 담겨져 있을 것이야...!
# 그게 page rank의 아이디어
# 많은 사람들이 들어가면 Node는 점수를 더 많이 얻어
# HITS는 edge의 개수로 node 점수 매겨
# random surfer는 cycle에 빠져버릴 수 있어 그러면 그 node들의 value가 펌핑이 되겠지 
# 들어오는 node는 없고 누군가를 가리키는 node만 있어 dangling node가 있을 수 있는거지
# page rank에서는 dangling node와 cycle를 자신만의 방법으로 해결함

## PageRank using NetworkX Library

In [ ]:
pageranks = nx.pagerank(g)

In [ ]:
pageranks

In [ ]:
plt.figure(figsize=[20, 15])
np.random.seed(5)
nx.draw_networkx(g, with_labels=True, node_color=list(pageranks.values()), node_size=np.array(list(pageranks.values()))*50000, font_size=10, font_color='white')
plt.show()

# Quiz_Rage Rank

In [ ]:
# Relabel nodes to perform matrix operations
g = nx.convert_node_labels_to_integers(g)
g.nodes

In [ ]:
# Build vectors
w = np.full(g.number_of_nodes(), 1.0 / g.number_of_nodes())
d = list(dict(g.out_degree).values())
d = np.array(d) == 0
w, d

## Sparse matrix

In [ ]:
# Build matrix M from the graph (as a sparse matrix)
from scipy.sparse import csr_matrix         # Compressed Sparse Rows
from sklearn.preprocessing import normalize
5
M = nx.to_scipy_sparse_matrix(g)
M = normalize(M, norm='l1', axis=0)
print(M)

In [ ]:
# nparray -> csr_matrix 변환
# 주의: 수식에서의 vector와 nparray는 축이 다름!
w = w[np.newaxis]
w = csr_matrix(w.T)
dt = csr_matrix(d)

In [ ]:
print(w)
print(dt)

In [ ]:
# (M + w x d^T)
MwdT = M + w.dot(dt)
# M 은 transition matrix
# column이 source, row가 destination
# 실제로는 random surfer가 방문할 확률을 구한다.
# w는 random surfing matrix
# d는 dangling matrix


## 아래 빈칸을 채워 페이지 랭크 알고리즘을 직접 작성해보시오

In [ ]:
# Initialization
max_iter = 50
eps = 0.000001
alpha = 0.15

In [ ]:
# 답변
"""Page rank main algorithm
max iter만큼 반복:
    rank < 이전 step의 rank를 edge를 따라 전파
    (1-alpha의 확률로 이전 step의 노드별 rank 전파, alpha의 확률로 random jump)
    (이전 step이 rank 전파: M*(l1 normalized rank), random jupmy: M+w)
    이전 step의 rank와 새로운 rank의 차이가 eps 미만일 때
        break
"""
ranks = np.full(g.number_of_nodes(), 1.0 / g.number_of_nodes())
ranks = ranks[np.newaxis]
ranks = csr_matrix(ranks.T)

for i in range(max_iter):
  new_ranks = (1.0 - alpha) * MwdT.dot(ranks) + alpha * w
  
  diff  = abs(new_ranks - ranks).sum()
  ranks = new_ranks
  
  if diff < eps:
    break
  


In [ ]:
# 결과를 dictionary로 변경
pageranks = dict(zip(g.nodes(), ranks.T.toarray()[0]))

In [ ]:
plt.figure(figsize=[20, 15])
np.random.seed(5)
nx.draw_networkx(g, with_labels=True, node_color=list(pageranks.values()), node_size=np.array(list(pageranks.values()))*65000, font_size=10, font_color='white')
plt.show()